In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint

In [2]:
aa_data = pd.read_csv('Data/AAdatabase.csv')

In [3]:
aa_data.head()

,Year,Ceremony,Award,Winner,Name,Film
0,1927/1928,1,Actor,NaN,Richard Barthelmess,The Noose
1,1927/1928,1,Actor,1.0,Emil Jannings,The Last Command
2,1927/1928,1,Actress,NaN,Louise Dresser,A Ship Comes In
3,1927/1928,1,Actress,1.0,Janet Gaynor,7th Heaven
4,1927/1928,1,Actress,NaN,Gloria Swanson,Sadie Thompson


In [4]:
aa_data = aa_data.fillna('Nominee')
aa_data = aa_data.replace({'Winner':{1:'Winner'}})
aa_data = aa_data.rename(columns={'Winner':'Result'})
aa_data = aa_data.drop('Year',axis=1)

In [5]:
aa_data.head()

,Ceremony,Award,Result,Name,Film
0,1,Actor,Nominee,Richard Barthelmess,The Noose
1,1,Actor,Winner,Emil Jannings,The Last Command
2,1,Actress,Nominee,Louise Dresser,A Ship Comes In
3,1,Actress,Winner,Janet Gaynor,7th Heaven
4,1,Actress,Nominee,Gloria Swanson,Sadie Thompson


---

In [6]:
# aa_data.to_json(path_or_buf='/AA.json',orient='index')

In [7]:
# pd.read_json('AA.json').transpose()

In [8]:
meta = pd.read_csv('Data/movies_metadata.csv')
kw = pd.read_csv('Data/keywords.csv')
meta = meta[meta['id'].str.contains('-') == False]
meta['id'] = pd.to_numeric(meta['id'])

In [9]:
combined = kw.merge(meta,how='inner',on='id')

In [10]:
combined = combined.drop(['adult','homepage','imdb_id','original_language','spoken_languages','status','tagline','video'],axis=1)

In [11]:
# combined.head()

In [12]:
# combined.to_json(r'/combined.json',orient='index')

---

In [13]:
from pymongo import MongoClient

In [14]:
conn = 'mongodb://localhost:27017'
client = MongoClient(conn)
db = client['movies_db']
col = db['awards']
col2 = db['movies']

dict_data = aa_data.to_dict(orient='records')

In [15]:
dict_data

[{'Ceremony': 1,
  'Award': 'Actor',
  'Result': 'Nominee',
  'Name': 'Richard Barthelmess',
  'Film': 'The Noose'},
 {'Ceremony': 1,
  'Award': 'Actor',
  'Result': 'Winner',
  'Name': 'Emil Jannings',
  'Film': 'The Last Command'},
 {'Ceremony': 1,
  'Award': 'Actress',
  'Result': 'Nominee',
  'Name': 'Louise Dresser',
  'Film': 'A Ship Comes In'},
 {'Ceremony': 1,
  'Award': 'Actress',
  'Result': 'Winner',
  'Name': 'Janet Gaynor',
  'Film': '7th Heaven'},
 {'Ceremony': 1,
  'Award': 'Actress',
  'Result': 'Nominee',
  'Name': 'Gloria Swanson',
  'Film': 'Sadie Thompson'},
 {'Ceremony': 1,
  'Award': 'Art Direction',
  'Result': 'Nominee',
  'Name': 'Rochus Gliese',
  'Film': 'Sunrise'},
 {'Ceremony': 1,
  'Award': 'Art Direction',
  'Result': 'Winner',
  'Name': 'William Cameron Menzies',
  'Film': 'The Dove; Tempest'},
 {'Ceremony': 1,
  'Award': 'Art Direction',
  'Result': 'Nominee',
  'Name': 'Harry Oliver',
  'Film': '7th Heaven'},
 {'Ceremony': 1,
  'Award': 'Cinematography

In [16]:
for x in dict_data:
    col.insert_one(x)

In [17]:
combined_dict = combined.to_dict(orient='records')
combined_dict

[{'id': 862,
  'keywords': "[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]",
  'belongs_to_collection': "{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",
  'budget': '30000000',
  'genres': "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",
  'original_title': 'Toy Story',
  'overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their 

In [18]:
for x in combined_dict:
    col2.insert_one(x)

# This is a fun game! Guess the movie based on keywords!

In [153]:
import random
import ast
from pprint import pprint

## Run this cell to generate keywords from a random movie

In [223]:
random_int = random.randint(0,45000)
random_movie = combined_dict[random_int]
keywords = random_movie['keywords']
title = random_movie['title']
release_date = random_movie['release_date']

key_dict = ast.literal_eval(keywords)
key_list = []

for i in key_dict:
    
    key_list.append(i['name'])
    
print(f'Keywords: {key_list}')

Keywords: ['retiree', 'old friends', 'aftercreditsstinger', 'duringcreditsstinger', 'ice fishing']


## Run this cell for the answer!

In [224]:
print(f'Movie: {title}')
print(f'Release: {release_date}')
print(f'Overview: {random_movie["overview"]}')

Movie: Grumpy Old Men
Release: 1993-12-25
Overview: For decades, next-door neighbors and former friends John and Max have feuded, trading insults and wicked pranks. When an attractive widow moves in nearby, their bad blood erupts into a high-stakes rivalry full of naughty jokes and adolescent hijinks. Will this love triangle destroy the two old grumps? Or will the geriatric odd couple overcome their differences and rediscover their friendship?
